In [1]:
# importing some useful libraries
%pylab inline

import pandas as pd
import time 
import joblib

from sklearn import metrics
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

from PIL import Image
rng = np.random.default_rng(12345)

In [2]:
MODEL_FILE = "../Dumps/model-combined-URLs.sav"
SCALER_FILE = "../Dumps/scaler-combined-URLs.sav"
ENCODER_FILE = "../Dumps/encoder-combined-URLs.sav"

URL_REGEX = ("^((?P<scheme>[^:/?#]+):(?=//))?(//)?(((?P<login>[^:]+)" + 
    "(?::(?P<password>[^@]+)?)?@)?(?P<host>[^@/?#:]*)(?::(?P<port>\d+)?)?)?" + 
    "(?P<path>[^?#]*)(\?(?P<query>[^#]*))?(#(?P<fragment>.*))?")

"^(?P<url>(?P<protocol>http[s]?|ftp):\/)?\/?(?P<host>[^:\/\s]+)(?P<path>(\/\w+)*\/)(?P<file>[\w\-\.]+[^#?\s]+)(?P<query>.*)?(?P<hash>#[\w\-]+)?$"

'^(?P<url>(?P<protocol>http[s]?|ftp):\\/)?\\/?(?P<host>[^:\\/\\s]+)(?P<path>(\\/\\w+)*\\/)(?P<file>[\\w\\-\\.]+[^#?\\s]+)(?P<query>.*)?(?P<hash>#[\\w\\-]+)?$'

# Wczytywanie danych

In [3]:
df = pd.read_csv("../Data/cleaned_combined_urls.csv", index_col=0)
df.head(2)

/home/pk/miniconda3/envs/Hackathon-2021-RTFD/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,url,label
0,?guid=Windows Updates Manager,1.0
1,69.162.100.198/,1.0


In [4]:
df["url"] = df.url.astype(str)

## Czyszczenie danych

In [5]:
print(df.shape)
print(df[df.url.str.count(".") > 0].shape)

(1958842, 2)
(1958842, 2)


In [6]:
url_parse_groups = ["scheme", 4, "host", "port", "path", "query", "fragment"]
df = pd.concat(
    [df, df.url.str.extract(URL_REGEX)[url_parse_groups]],
    axis=1
)    
print(df.shape)
df.columns = ["url", "label", "scheme", 
              "user", "host", "port", 
              "path", "query", "fragment"]
df.head()

(1958842, 9)


,url,label,scheme,user,host,port,path,query,fragment
0,?guid=Windows Updates Manager,1.0,NaN,NaN,,NaN,,guid=Windows Updates Manager,NaN
1,69.162.100.198/,1.0,NaN,NaN,69.162.100.198,NaN,/,NaN,NaN
2,babicz123.ddns.net/,1.0,NaN,NaN,babicz123.ddns.net,NaN,/,NaN,NaN
3,highpowerresources.com,1.0,NaN,NaN,highpowerresources.com,NaN,,NaN,NaN
4,intent.nofrillspace.com/users/web11_focus/380...,1.0,NaN,NaN,intent.nofrillspace.com,NaN,/users/web11_focus/3807/space.gif,NaN,NaN


In [7]:
df.drop(np.where(df.host.str.len() == 0)[0], inplace=True)

In [8]:
df.groupby("label").count()

,url,scheme,user,host,port,path,query,fragment
label,,,,,,,,
0.0,1725805,963664,0,1725805,182,1725805,19340,518
1.0,233036,154514,0,233036,1121,233036,10188,135


In [9]:
from pathlib import Path
df["filename"] = [Path(x).stem for x in  df["path"]]
df["fileext"] = [Path(x).suffix.replace('.', '') for x in  df["path"]]

In [10]:
df.head(2)

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext
1,69.162.100.198/,1.0,NaN,NaN,69.162.100.198,NaN,/,NaN,NaN,,
2,babicz123.ddns.net/,1.0,NaN,NaN,babicz123.ddns.net,NaN,/,NaN,NaN,,


## Obliczanie metryk

### Popularność domeny

In [11]:
df['tld'] = df.host.str.split(".").apply(lambda x : x[-1])
count = df.groupby(["tld"]).count().reset_index().iloc[:, [0,1]]
count.columns = ["tld", "count"]
df = df.merge(count, on=["tld"])
df.head(10)

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count
0,69.162.100.198/,1.0,NaN,NaN,69.162.100.198,NaN,/,NaN,NaN,,,198,11
1,100.42.48.198/~fruiti/js/prototype/windows/the...,1.0,NaN,NaN,100.42.48.198,NaN,/~fruiti/js/prototype/windows/themes/default/f...,NaN,NaN,express,html,198,11
2,185.66.9.198/module/09e2a5987534a3f12a01504e28...,1.0,NaN,NaN,185.66.9.198,NaN,/module/09e2a5987534a3f12a01504e2823f826/,NaN,NaN,09e2a5987534a3f12a01504e2823f826,,198,11
3,193.238.152.198/OeeC,1.0,NaN,NaN,193.238.152.198,NaN,/OeeC,NaN,NaN,OeeC,,198,11
4,http://209.150.104.198/horror/julieshiel/,0.0,http,NaN,209.150.104.198,NaN,/horror/julieshiel/,NaN,NaN,julieshiel,,198,11
5,216.22.34.198/uploadform/server/php/files/GD/i...,1.0,NaN,NaN,216.22.34.198,NaN,/uploadform/server/php/files/GD/index.php,NaN,NaN,index,php,198,11
6,37.187.65.198/Chrome_Update_2014.exe,1.0,NaN,NaN,37.187.65.198,NaN,/Chrome_Update_2014.exe,NaN,NaN,Chrome_Update_2014,exe,198,11
7,5.200.52.198/flex/3435dde3r.exe,1.0,NaN,NaN,5.200.52.198,NaN,/flex/3435dde3r.exe,NaN,NaN,3435dde3r,exe,198,11
8,91.239.24.198:6892,1.0,NaN,NaN,91.239.24.198,6892,,NaN,NaN,,,198,11
9,91.239.25.198:6892,1.0,NaN,NaN,91.239.25.198,6892,,NaN,NaN,,,198,11


In [12]:
"""This module is used to calculate Shannon entropy of objects"""
import re
from math import log2

# Propability of letter occuring in dataset
# for more info check '/Notebooks/Entropy.ipynb'
prop_of_letters = {'g': 0.017823195816706855,
                   'u': 0.019656863921083692,
                   'i': 0.04282234162349053,
                   'd': 0.02362298601231347,
                   'w': 0.06868367734904952,
                   'n': 0.04192416654368707,
                   'o': 0.05917547697301064,
                   's': 0.04087732549559563,
                   'p': 0.047038663571700494,
                   'a': 0.04921073758433189,
                   't': 0.07547396818911765,
                   'e': 0.05859317164000978,
                   'm': 0.03821830076858004,
                   'r': 0.03626613607334285,
                   '6': 0.005261345563926245,
                   '9': 0.004489300290958807,
                   '.': 0.06449691896747548,
                   '1': 0.0074827325750036165,
                   '2': 0.007503935952713802,
                   '0': 0.006426077392086318,
                   '8': 0.00587551654457156,
                   '/': 0.05952224307019093,
                   'b': 0.018622381413143332,
                   'c': 0.049838478726711724,
                   'z': 0.0043048914859593656,
                   '3': 0.0066693710060409025,
                   'h': 0.033948061659907026,
                   'f': 0.014128719284484144,
                   'l': 0.03218672736406151,
                   '7': 0.004617732178803359,
                   '%': 0.0009838367257526048,
                   '5': 0.0058310500324593425,
                   '4': 0.006094335402540674,
                   'j': 0.0035587749148775255,
                   'y': 0.0096172463185484,
                   'x': 0.006756123111415147,
                   "'": 0.00032895525990373425,
                   'v': 0.007133179748181417,
                   '\\': 0.0007063753831450352,
                   '&': 0.0015393652217594634,
                   ';': 0.0008533450812162063,
                   'q': 0.0019404119658778284,
                   'k': 0.009749313071143269,
                   '(': 1.0662269934264683e-05,
                   ')': 1.0783432092608599e-05,
                   ',': 9.123510523296938e-05,
                   '#': 1.853781022661928e-05,
                   '[': 2.7867296419100876e-06,
                   ']': 2.7867296419100876e-06,
                   '*': 4.361837700381007e-06,
                   '$': 5.088810650444507e-06}

# Filter all characters present in dictionary
cleaner = re.compile(r"""[^a-z0-9/;,.\'\[\]@&%1#$*()\\]+""")

def clean(test):
    return cleaner.sub('', test)

def entropy2(text):
    ent = 0.0
    for letter in clean(text):
        ent += prop_of_letters[letter] * log2(prop_of_letters[letter])
    return ent

import math


def entropy(string):
    "Calculates the Shannon entropy of a string"

    # get probability of chars in string
    prob = [ float(string.count(c)) / len(string) for c in dict.fromkeys(list(string)) ]

    # calculate the entropy
    entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob ])

    return entropy

In [13]:
def character_continiuity_rate(text):
    digits  = max(map(len,re.findall("\d+", text)), default = 0)
    chars = max(map(len,re.findall("[a-z]+", text)), default = 0)
    special_chars = max(map(len,re.findall("[.\+\-\,\&\/]+", text)), default = 0)
    return (digits + chars + special_chars) /(len(text) + 1)

### Pozostałe

In [14]:
def calculate_metrics(df_):
    ##### Scheme
    scheme = df_.scheme
    scheme = scheme.fillna("n")
    scheme = scheme.replace({
        "[nh]+t+p+s+": 0, 
        "[nh]+t+p*(?![a-z]*s)" : 1,
        "n": 0.5, 
        "none": 0.5,
        "\w+" : 0.5
    }, regex=True)
    scheme = scheme.astype('float64')    
    
    #### URL
    url_len = df_.url.str.len()
    url_entropy = np.array([entropy(h) for h in df_.url.to_numpy()])
    url_entropy2 = np.array([entropy2(h) for h in df_.url.to_numpy()])
    
    #### Query
    
    
    #### Host
    host = df_.host
    host_len = host.str.len()
    host_subdomains_count = host.str.count(".")
    #host_subdomains_mean_len = (host_len - host_subdomains_count) / host_subdomains_count
    host_digit = host.str.count("\d")
    host_nspecial = host.str.count("([^A-Za-z\d\s])")
    host_first_len = host.str.split(".", n=1, expand=True)[0].str.len() / host_len
    host_first_count_tld_in_name = host.str.split(".", n=1, expand=True)[0].str.count("com|pl|gov")
    #host_digit_letter_count = host.str.count("[A-Za-z]\d|\d[A-Za-z]")
    host_has_a = host.str.count("@")
    host_pref_suf_number = host.str.count("[-_]")
    host_is_ipv4 = host.str.count("(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)" + 
                                  "(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}")
    host_entropy = np.array([entropy(h) for h in host.to_numpy()])
    
    
    #### Path
    path = df_.path
    path_len = path.str.len()
    path_digit = path.str.count("\d")
    path_entropy = np.array([entropy(h) for h in path.to_numpy()])
    
    #### Filename
    filename = df_.filename
    filename_len = filename.str.len()
    filename_digit = filename.str.count("\d")
    filename_entropy = np.array([entropy(h) for h in filename.to_numpy()])
    
    #### File extension
    df_.loc[df_["fileext"].str.len() > 4,"fileext"] = ''
    fileext = df_.fileext
    fileext_exe = fileext.str.contains("exe|jar|run", na=False)
    fileext_digit= filename.str.count("\d")
    fileext_len = filename.str.len()
    
    #### Cross 
    path_url_ratio = path_len / url_len
    host_url_ratio = host_len / url_len
    host_path_ratio = host_len / path_len
    
    ccr = host.apply(lambda x: character_continiuity_rate(x[:x.find('.')]))
    
    return np.array([
        host_len, host_subdomains_count, 
        host_digit, host_nspecial, host_first_len, 
        host_pref_suf_number, host_is_ipv4, host_entropy, path_len,
        url_len, path_url_ratio, host_url_ratio, path_digit, filename_len, filename_digit,
        fileext_exe, fileext_digit, fileext_len, ccr, path_entropy, filename_entropy, host_first_count_tld_in_name, url_entropy
    ]).T

In [15]:
fnames = ["host_len",
"host_subdomains_count",
"host_digit",
"host_nspecial",
"host_first_len",
"host_pref_suf_number",
"host_is_ipv4",
"host_entropy",
"path_len",
"url_len",
"path_url_ratio",
"host_url_ratio",
"path_digit",
"filename_len",
"filename_digit",
"fileext_exe",
"fileext_digit",
"fileext_len",
"ccr",
"path_entropy",
"filename_entropy", 
"host_first_count_tld_in_name", 
'url_entropy']

In [17]:
df[["url", "scheme"]].groupby("scheme").count()

,url
scheme,
http,1
ftp,207
gopher,18
hXXp,24
hXXps,2
htpp,1
http,1106003
https,11888
https.portal.apple.com.idmswebauth.login.html.appidkey.05c7e09b5896b0334b3af1139274f266b2hxxp,1


# Zapisanie przetworzonych danych

In [18]:
df

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count
0,69.162.100.198/,1.0,NaN,NaN,69.162.100.198,NaN,/,NaN,NaN,,,198,11
1,100.42.48.198/~fruiti/js/prototype/windows/the...,1.0,NaN,NaN,100.42.48.198,NaN,/~fruiti/js/prototype/windows/themes/default/f...,NaN,NaN,express,html,198,11
2,185.66.9.198/module/09e2a5987534a3f12a01504e28...,1.0,NaN,NaN,185.66.9.198,NaN,/module/09e2a5987534a3f12a01504e2823f826/,NaN,NaN,09e2a5987534a3f12a01504e2823f826,,198,11
3,193.238.152.198/OeeC,1.0,NaN,NaN,193.238.152.198,NaN,/OeeC,NaN,NaN,OeeC,,198,11
4,http://209.150.104.198/horror/julieshiel/,0.0,http,NaN,209.150.104.198,NaN,/horror/julieshiel/,NaN,NaN,julieshiel,,198,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958836,xn--qxafk0d.xn--qxam,0.0,NaN,NaN,xn--qxafk0d.xn--qxam,NaN,,NaN,NaN,,,xn--qxam,2
1958837,xn--uxaakbl0acb.xn--qxam,0.0,NaN,NaN,xn--uxaakbl0acb.xn--qxam,NaN,,NaN,NaN,,,xn--qxam,2
1958838,xn--s5bs5fmh.xn--54b7fta0cc,0.0,NaN,NaN,xn--s5bs5fmh.xn--54b7fta0cc,NaN,,NaN,NaN,,,xn--54b7fta0cc,1
1958839,http://xpressparcel.net 27048,1.0,http,NaN,xpressparcel.net 27048,NaN,,NaN,NaN,,,net 27048,1


In [19]:
X = calculate_metrics(df)

In [20]:
X

array([[15.        , 15.        , 11.        , ..., -0.        ,
         0.        ,  3.03063906],
       [13.        , 13.        , 10.        , ...,  2.23592635,
         0.        ,  4.65131884],
       [12.        , 12.        ,  9.        , ...,  3.58174756,
         0.        ,  4.13351492],
       ...,
       [27.        , 27.        ,  6.        , ..., -0.        ,
         0.        ,  3.83803982],
       [22.        , 22.        ,  5.        , ..., -0.        ,
         0.        ,  4.15919953],
       [20.        , 20.        ,  0.        , ...,  2.91397707,
         0.        ,  4.07565526]])

In [21]:
calculated_df = pd.DataFrame(X, columns=fnames)

In [22]:
calculated_df

,host_len,host_subdomains_count,host_digit,host_nspecial,host_first_len,host_pref_suf_number,host_is_ipv4,host_entropy,path_len,url_len,...,filename_len,filename_digit,fileext_exe,fileext_digit,fileext_len,ccr,path_entropy,filename_entropy,host_first_count_tld_in_name,url_entropy
0,15.0,15.0,11.0,3.0,0.200000,0.0,1.0,2.872906,1.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.500000,-0.000000,-0.000000,0.0,3.030639
1,13.0,13.0,10.0,3.0,0.230769,0.0,1.0,2.719295,67.0,80.0,...,7.0,0.0,0.0,0.0,7.0,0.750000,4.310138,2.235926,0.0,4.651319
2,12.0,12.0,9.0,3.0,0.250000,0.0,1.0,2.522055,41.0,53.0,...,32.0,25.0,0.0,25.0,32.0,0.750000,4.067458,3.581748,0.0,4.133515
3,15.0,15.0,12.0,3.0,0.200000,0.0,1.0,2.739572,5.0,20.0,...,4.0,0.0,0.0,0.0,4.0,0.750000,1.921928,1.500000,0.0,3.346439
4,15.0,15.0,12.0,3.0,0.200000,0.0,1.0,2.822580,19.0,41.0,...,10.0,0.0,0.0,0.0,10.0,0.750000,3.221097,2.721928,0.0,4.201843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958836,20.0,20.0,1.0,5.0,0.550000,4.0,0.0,3.221928,0.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.666667,-0.000000,-0.000000,0.0,3.221928
1958837,24.0,24.0,1.0,5.0,0.625000,4.0,0.0,3.418296,0.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.625000,-0.000000,-0.000000,0.0,3.418296
1958838,27.0,27.0,6.0,5.0,0.444444,4.0,0.0,3.838040,0.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.461538,-0.000000,-0.000000,0.0,3.838040
1958839,22.0,22.0,5.0,1.0,0.545455,0.0,0.0,3.970573,0.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.923077,-0.000000,-0.000000,0.0,4.159200


In [25]:
join = pd.concat([df, calculated_df],axis=1, join="inner")
join

,url,label,scheme,user,host,port,path,query,fragment,filename,...,filename_len,filename_digit,fileext_exe,fileext_digit,fileext_len,ccr,path_entropy,filename_entropy,host_first_count_tld_in_name,url_entropy
0,69.162.100.198/,1.0,NaN,NaN,69.162.100.198,NaN,/,NaN,NaN,,...,0.0,0.0,0.0,0.0,0.0,0.500000,-0.000000,-0.000000,0.0,3.030639
1,100.42.48.198/~fruiti/js/prototype/windows/the...,1.0,NaN,NaN,100.42.48.198,NaN,/~fruiti/js/prototype/windows/themes/default/f...,NaN,NaN,express,...,7.0,0.0,0.0,0.0,7.0,0.750000,4.310138,2.235926,0.0,4.651319
2,185.66.9.198/module/09e2a5987534a3f12a01504e28...,1.0,NaN,NaN,185.66.9.198,NaN,/module/09e2a5987534a3f12a01504e2823f826/,NaN,NaN,09e2a5987534a3f12a01504e2823f826,...,32.0,25.0,0.0,25.0,32.0,0.750000,4.067458,3.581748,0.0,4.133515
3,193.238.152.198/OeeC,1.0,NaN,NaN,193.238.152.198,NaN,/OeeC,NaN,NaN,OeeC,...,4.0,0.0,0.0,0.0,4.0,0.750000,1.921928,1.500000,0.0,3.346439
4,http://209.150.104.198/horror/julieshiel/,0.0,http,NaN,209.150.104.198,NaN,/horror/julieshiel/,NaN,NaN,julieshiel,...,10.0,0.0,0.0,0.0,10.0,0.750000,3.221097,2.721928,0.0,4.201843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958836,xn--qxafk0d.xn--qxam,0.0,NaN,NaN,xn--qxafk0d.xn--qxam,NaN,,NaN,NaN,,...,0.0,0.0,0.0,0.0,0.0,0.666667,-0.000000,-0.000000,0.0,3.221928
1958837,xn--uxaakbl0acb.xn--qxam,0.0,NaN,NaN,xn--uxaakbl0acb.xn--qxam,NaN,,NaN,NaN,,...,0.0,0.0,0.0,0.0,0.0,0.625000,-0.000000,-0.000000,0.0,3.418296
1958838,xn--s5bs5fmh.xn--54b7fta0cc,0.0,NaN,NaN,xn--s5bs5fmh.xn--54b7fta0cc,NaN,,NaN,NaN,,...,0.0,0.0,0.0,0.0,0.0,0.461538,-0.000000,-0.000000,0.0,3.838040
1958839,http://xpressparcel.net 27048,1.0,http,NaN,xpressparcel.net 27048,NaN,,NaN,NaN,,...,0.0,0.0,0.0,0.0,0.0,0.923077,-0.000000,-0.000000,0.0,4.159200


In [28]:
join.to_csv("../Data/data_for_presentation.xz", compression="xz")

# Trenowanie modelu

In [21]:
sc = StandardScaler()
dane = df
label_distribution = df.groupby("label").count()
label_distribution

,url,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count
label,,,,,,,,,,,,
0.0,1725805,963664,0,1725805,182,1725805,19340,518,1725805,1725805,1725805,1725805
1.0,233036,154514,0,233036,1121,233036,10188,135,233036,233036,233036,233036


In [22]:
df[df["query"].notna()]

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count
12,'64835108.net/images/?us.battle.net/login/en/r...,1.0,NaN,NaN,'64835108.net,NaN,/images/,us.battle.net/login/en/ref=http\%3A\%2F\%2Fus....,NaN,images,,net,90987
15,'antispam5.rilinfo.net/Message?action=Detail&a...,1.0,NaN,NaN,'antispam5.rilinfo.net,NaN,/Message,action=Detail&amp;mid=219276427&amp;h=afdd9b90...,NaN,Message,,net,90987
18,'bulletpens.net/OScommerce/No/loginAction.acti...,1.0,NaN,NaN,'bulletpens.net,NaN,/OScommerce/No/loginAction.action.php,cmd=verification&appId=ABONNESNEUF&api=\%3C?ec...,NaN,loginAction.action,php,net,90987
19,'chao-yue.net/images/?us.battle.net/login/en/?...,1.0,NaN,NaN,'chao-yue.net,NaN,/images/,us.battle.net/login/en/?ref=http\%3A\%2F\%2Fus...,NaN,images,,net,90987
20,'cielopremiosnet.site90.net/xx/cadastro.html?i...,1.0,NaN,NaN,'cielopremiosnet.site90.net,NaN,/xx/cadastro.html,ios\%20autenticado&amp;pbx=1&amp;oq=&amp;aq=&a...,NaN,cadastro,html,net,90987
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957739,netsync2/post.php?filename=,1.0,NaN,NaN,netsync2,NaN,/post.php,filename=,NaN,post,php,netsync2,1
1957803,http://www.oyster.net.ck/about/index.php?about...,0.0,http,NaN,www.oyster.net.ck,NaN,/about/index.php,about=domain,NaN,index,php,ck,7
1957826,products/add_user.php?name=,1.0,NaN,NaN,products,NaN,/add_user.php,name=,NaN,add_user,php,products,1
1957827,qaz.MACLARENREPLACEMENTSTROLLERBUGGYPUSHCHAIRP...,1.0,NaN,NaN,qaz.MACLARENREPLACEMENTSTROLLERBUGGYPUSHCHAIRP...,NaN,/,q=zn_QMvXcJwDQDoPGMvrESLtEMU3QA0KK2OH_76yyEoH9...,NaN,,,UK,1


In [23]:
balanced_df_half_size = 130000
good_indices = rng.integers(0, label_distribution.url[0]-1, balanced_df_half_size)
bad_indices = rng.integers(0, label_distribution.url[1]-1, balanced_df_half_size)

balanced_df = pd.concat(
    [dane[dane.label == 0].iloc[good_indices],
     dane[dane.label == 1].iloc[bad_indices]])
le = preprocessing.LabelEncoder()
le.fit(balanced_df.label)
balanced_df["enc"] = le.transform(balanced_df.label)
balanced_df

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count,enc
1368471,http://www.gnucash.org/,0.0,http,NaN,www.gnucash.org,NaN,/,NaN,NaN,,,org,158316,0
424711,naturaily.com,0.0,NaN,NaN,naturaily.com,NaN,,NaN,NaN,,,com,1186983,0
1545193,house.leg.state.mn.us/members/members.asp?id=1...,0.0,NaN,NaN,house.leg.state.mn.us,NaN,/members/members.asp,id=15356,NaN,members,asp,us,8597,0
587045,trendsbuzz.com/n-cornelia+marie+devlin.html,0.0,NaN,NaN,trendsbuzz.com,NaN,/n-cornelia+marie+devlin.html,NaN,NaN,n-cornelia+marie+devlin,html,com,1186983,0
382371,luxyride.com,0.0,NaN,NaN,luxyride.com,NaN,,NaN,NaN,,,com,1186983,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787174,http://www.del-cargo.com,1.0,http,NaN,www.del-cargo.com,NaN,,NaN,NaN,,,com,1186983,1
935179,http://www.jfs-transport.com,1.0,http,NaN,www.jfs-transport.com,NaN,,NaN,NaN,,,com,1186983,1
810354,http://www.easylink-logistics.com,1.0,http,NaN,www.easylink-logistics.com,NaN,,NaN,NaN,,,com,1186983,1
688876,http://www.barclaysbnkonline.com,1.0,http,NaN,www.barclaysbnkonline.com,NaN,,NaN,NaN,,,com,1186983,1


In [24]:
features = calculate_metrics(balanced_df)
features

array([[15.        , 15.        ,  0.        , ..., -0.        ,
         0.        ,  3.76226739],
       [13.        , 13.        ,  0.        , ..., -0.        ,
         0.        ,  3.54659356],
       [21.        , 21.        ,  0.        , ...,  2.23592635,
         0.        ,  4.19624538],
       ...,
       [26.        , 26.        ,  0.        , ..., -0.        ,
         0.        ,  4.16501382],
       [25.        , 25.        ,  0.        , ..., -0.        ,
         0.        ,  4.20281953],
       [14.        , 14.        ,  0.        , ..., -0.        ,
         0.        ,  3.78494183]])

In [25]:
trainX, testX, trainY, testY = train_test_split(features, balanced_df.enc, test_size=0.2)
trainX = sc.fit_transform(trainX)
testX = sc.transform(testX)

# Wektory wspierające

In [26]:
#from sklearn import svm

#clf = svm.SVC(kernel="linear")
#clf.fit(trainX, trainY)

In [27]:
#clf.score(testX, testY)

In [28]:
#clf.score(testX[testY == 1], testY[testY == 1]) #Dane dobre = 1

In [29]:
#clf.score(testX[testY == 0], testY[testY == 0]) # Dane złe = 1

# Drzewa losowe

In [30]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(random_state=1)
clf2.fit(trainX, trainY)

RandomForestClassifier(random_state=1)

In [31]:
clf2.score(testX, testY)

0.891576923076923

In [36]:
for c,v in zip(clf2.feature_importances_, fnames):
    print("{0}: {1}".format(c,v))

0.020686928476752506: host_len
0.018826844270696433: host_subdomains_count
0.026102402460599715: host_digit
0.03574496276338389: host_nspecial
0.08660194412792177: host_first_len
0.02295990170725838: host_pref_suf_number
0.0018928524332664414: host_is_ipv4
0.06086469405740673: host_entropy
0.12170207013242898: path_len
0.055531592185987376: url_len
0.15039985591492527: path_url_ratio
0.10079851052344345: host_url_ratio
0.007099136775149588: path_digit
0.03540472965980517: filename_len
0.003738525993731919: filename_digit
0.000665716873654451: fileext_exe
0.0048440531069387765: fileext_digit
0.029463205273089366: fileext_len
0.05429642838972651: ccr
0.03275331648046869: path_entropy
0.02275075098130903: filename_entropy
0.0024971629959273053: host_first_count_tld_in_name
0.10437441441612819: url_entropy


In [33]:
predY = clf2.predict(testX)

In [34]:
print(confusion_matrix(testY, predY))

[[22011  3958]
 [ 1680 24351]]


In [35]:
print(classification_report(testY, predY))

              precision    recall  f1-score   support

           0       0.93      0.85      0.89     25969
           1       0.86      0.94      0.90     26031

    accuracy                           0.89     52000
   macro avg       0.89      0.89      0.89     52000
weighted avg       0.89      0.89      0.89     52000



# Testowanie klasyfikatorów

In [64]:
def transform(url):
    df_ = pd.DataFrame([url], columns = ['url'])
    df_ = pd.concat(
        [df_, df_.url.str.extract(URL_REGEX)[url_parse_groups]],
        axis=1
    )
    
    df_.columns = ["url", "scheme", 
                  "user", "host", "port", 
                  "path", "query", "fragment"]
    
    df_["filename"] = [Path(x).stem for x in  df_["path"]]
    df_["fileext"] = [Path(x).suffix.replace('.', '') for x in  df_["path"]]
    
    return df_    

In [65]:
transform("https://allegro.pl")

,url,scheme,user,host,port,path,query,fragment,filename,fileext
0,https://allegro.pl,https,NaN,allegro.pl,NaN,,NaN,NaN,,


In [66]:
df.iloc[0:1000,:].apply(lambda x: str(x), axis=1)

0      url          69.162.100.198/\nlabel           ...
1      url         100.42.48.198/~fruiti/js/prototype...
2      url         185.66.9.198/module/09e2a5987534a3...
3      url         193.238.152.198/OeeC\nlabel       ...
4      url         http://209.150.104.198/horror/juli...
                             ...                        
995    url         http://acdk.sourceforge.net/\nlabe...
996    url         aceconomy.net\nlabel              ...
997    url         acehjobs.net\nlabel               ...
998    url         acestream.net\nlabel              ...
999    url         acestreamsearch.net\nlabel        ...
Length: 1000, dtype: object

In [67]:
balanced_df["pred"] = clf2.predict(sc.transform(features))

In [68]:
balanced_df[(balanced_df["enc"] == 1) & (balanced_df["pred"] == 0)] # Bad predicted as good

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count,enc,pred
744141,http://www.cheltsglssts.com,1.0,http,NaN,www.cheltsglssts.com,NaN,,NaN,NaN,,,com,1186983,1,0
1527555,http://www.finmurano.biz,1.0,http,NaN,www.finmurano.biz,NaN,,NaN,NaN,,,biz,5419,1,0
1126800,http://www.scommbuk.com,1.0,http,NaN,www.scommbuk.com,NaN,,NaN,NaN,,,com,1186983,1,0
1246729,http://www.whitelanec.com,1.0,http,NaN,www.whitelanec.com,NaN,,NaN,NaN,,,com,1186983,1,0
906921,http://www.hsgroupsonline.com,1.0,http,NaN,www.hsgroupsonline.com,NaN,,NaN,NaN,,,com,1186983,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774738,http://www.ctfs-online.com,1.0,http,NaN,www.ctfs-online.com,NaN,,NaN,NaN,,,com,1186983,1,0
1903455,http://www.pioneerbuyers.pro,1.0,http,NaN,www.pioneerbuyers.pro,NaN,,NaN,NaN,,,pro,1286,1,0
810695,http://www.ebarclayint.com,1.0,http,NaN,www.ebarclayint.com,NaN,,NaN,NaN,,,com,1186983,1,0
1655171,http://www.road.cargo.office.ms,1.0,http,NaN,www.road.cargo.office.ms,NaN,,NaN,NaN,,,ms,204,1,0


In [69]:
balanced_df[(balanced_df["enc"] == 0) & (balanced_df["pred"] == 1)] # Good predicted as bad

,url,label,scheme,user,host,port,path,query,fragment,filename,fileext,tld,count,enc,pred
1887732,dogrucan.com.tr,0.0,NaN,NaN,dogrucan.com.tr,NaN,,NaN,NaN,,,tr,8598,0,1
1663129,cribsforbabies.tk/Bowser/Bowser-Jr/,0.0,NaN,NaN,cribsforbabies.tk,NaN,/Bowser/Bowser-Jr/,NaN,NaN,Bowser-Jr,,tk,1396,0,1
20637,http://jffnms.sourceforge.net,0.0,http,NaN,jffnms.sourceforge.net,NaN,,NaN,NaN,,,net,90987,0,1
1293819,farmingdaleschools.org/fps/AAE.cfm,0.0,NaN,NaN,farmingdaleschools.org,NaN,/fps/AAE.cfm,NaN,NaN,AAE,cfm,org,158316,0,1
719776,http://www.btathletics.com,0.0,http,NaN,www.btathletics.com,NaN,,NaN,NaN,,,com,1186983,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1393402,http://www.nccpr.org,0.0,http,NaN,www.nccpr.org,NaN,,NaN,NaN,,,org,158316,0,1
986094,http://www.marcusgunther.com,0.0,http,NaN,www.marcusgunther.com,NaN,,NaN,NaN,,,com,1186983,0,1
650065,http://www.alabasterlighting-bali.com,0.0,http,NaN,www.alabasterlighting-bali.com,NaN,,NaN,NaN,,,com,1186983,0,1
638666,http://www.aceofbase.com,0.0,http,NaN,www.aceofbase.com,NaN,,NaN,NaN,,,com,1186983,0,1


# Podsumowanie
Model w obecnym stanie jest w stanie rozpoznać proste podmiany i działa szybko, jak dla mnie nadaje się do wrzucenia.

In [75]:
clf2.decision_path(sc.transform(calculate_metrics(transform("https://allegro.pl"))))

(<1x4595220 sparse matrix of type '<class 'numpy.int64'>'
 	with 1603 stored elements in Compressed Sparse Row format>,
 array([      0,   46871,   92662,  138223,  184698,  231221,  277156,
         322777,  368486,  414447,  459488,  505673,  551780,  597899,
         644590,  690811,  736600,  782245,  828672,  874677,  920102,
         966137, 1012132, 1058277, 1103918, 1150027, 1195876, 1241719,
        1287524, 1333687, 1379330, 1424711, 1470338, 1516515, 1562560,
        1608005, 1654172, 1700445, 1746460, 1792667, 1838838, 1885297,
        1930438, 1975735, 2021714, 2068023, 2114310, 2160343, 2206162,
        2251465, 2297428, 2343421, 2389260, 2435175, 2480932, 2527907,
        2574580, 2620525, 2666726, 2712205, 2758112, 2804371, 2850110,
        2895779, 2941556, 2987277, 3032756, 3078673, 3124396, 3170367,
        3216152, 3262295, 3307272, 3353585, 3400140, 3446067, 3491028,
        3536713, 3582486, 3629425, 3675668, 3721655, 3767522, 3813489,
        3859502, 3904939, 39

In [76]:
clf2.decision_path(sc.transform(calculate_metrics(transform("https://a11egro.pl"))))

(<1x4595220 sparse matrix of type '<class 'numpy.int64'>'
 	with 1331 stored elements in Compressed Sparse Row format>,
 array([      0,   46871,   92662,  138223,  184698,  231221,  277156,
         322777,  368486,  414447,  459488,  505673,  551780,  597899,
         644590,  690811,  736600,  782245,  828672,  874677,  920102,
         966137, 1012132, 1058277, 1103918, 1150027, 1195876, 1241719,
        1287524, 1333687, 1379330, 1424711, 1470338, 1516515, 1562560,
        1608005, 1654172, 1700445, 1746460, 1792667, 1838838, 1885297,
        1930438, 1975735, 2021714, 2068023, 2114310, 2160343, 2206162,
        2251465, 2297428, 2343421, 2389260, 2435175, 2480932, 2527907,
        2574580, 2620525, 2666726, 2712205, 2758112, 2804371, 2850110,
        2895779, 2941556, 2987277, 3032756, 3078673, 3124396, 3170367,
        3216152, 3262295, 3307272, 3353585, 3400140, 3446067, 3491028,
        3536713, 3582486, 3629425, 3675668, 3721655, 3767522, 3813489,
        3859502, 3904939, 39

# Zapisanie modelu

In [72]:
import joblib

joblib.dump(clf, MODEL_FILE) # SVM
joblib.dump(sc, SCALER_FILE) # StandardScaler
joblib.dump(le, ENCODER_FILE) # LabelEncoder

['../Dumps/encoder-combined-URLs.sav']